#Assignment 5




In [0]:
using Pkg
Pkg.add("CuArrays")
Pkg.add("CUDAnative")
Pkg.add("CUDAdrv")
using CUDAnative, CUDAdrv, CuArrays

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [0]:
Pkg.add("BenchmarkTools")
Pkg.add("Test")
using BenchmarkTools, Test

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


##Part 1: for <=1024 elements

In [0]:
N=parse(Int32, readline())
a=rand(Int64, N)
d_a=CuArray(a)
range=Int32(ceil(log2(N)));

stdin> 1024


In [0]:
function adder!(d_a, dist)
    x=threadIdx().x
    if(x>dist)
      @inbounds d_a[x]+=d_a[x-dist]
    end      
    return nothing
end

adder! (generic function with 2 methods)

In [0]:
for i in 0:(range-1)
    dist=2^i
    @cuda threads=N adder!(d_a, dist)
    synchronize()
end
println(d_a)

[-2547971311437355441, -4648545475950504040, 5672021222267771124, -4796956356956310682, -7563984870538312765, -2536752669414602580, 6865440330673380784, -3385739755107118395, -4799409499151643136, -462486190851642095, 1160007513536810023, 716731105252001488, 1525305616813742006, -3713051394606287246, -7640052590966535256, 5447899397315516676, 573953899705946850, -977996670531674106, -2797531474554025826, -7282241986919716621, -1120155688330328504, 2016045922411070106, 3596875489908629228, 995796935076595400, 2496460492953333105, -4154357613550834503, -5195190868840641047, -6649995158728328145, 5762046358252452066, 7143405362029007827, 4094498260118903953, -5927618312577508785, 3148738893625965663, 8664129344808091794, 8407249210279932696, -2699257819265146055, 7042753309734290849, 249768660036878542, -115684026013614832, -5579070725478499233, -7057775036747328437, 8327384266855810289, 100782904241279402, 8213984778426687830, -5899199748865664903, 2853303323674992005, -11063381745720653

In [0]:
function summer!(a, N)
  for i in 2:N
    a[i]+=a[i-1]
  end
end

summer! (generic function with 1 method)

In [0]:
b=Array(d_a)
summer!(a,N)
@test isapprox(b[N], a[N])

Test Passed

##Part 2: for >1024 elements.
I don't think the kernel I made works for >1024 elements. I tried to make a function that accesses and adds elements that may be of different blocks, and it returned the same error code as when I tried to make a block adaptation of the kernel.

In [0]:
#making the add_in_place func
function add_in_place!(d_a)
    tidx=threadIdx().x
    bidx=blockIdx().x
    x=tidx + blockDim().x * (bidx-1)
    if(x%2==0)
        @inbounds d_a[x]+=d_a[x-1]
    end
    return nothing
end

add_in_place! (generic function with 1 method)

In [0]:
N=2048
a=rand(Int32, N)
d_a=CuArray(a);

In [0]:
temp=ceil(log2(N))
range=Int(temp)
for i in 0:(range-1)
    dist=2^i
    @cuda blocks=ceil(N/1024) threads=1024 add_in_place!(d_a)
    synchronize()
    d_a=d_s
end
println(d_a)

TypeError: ignored

In [0]:
#Block adaptation
N=parse(Int32, readline())
a=ones(Int32, N)
d_a=CuArray(a);

stdin> 2048


In [0]:
function adder!(d_a, dist)
    tx=threadIdx().x
    bx=blockIdx().x
    x=tx+blockDim().x*(bx-1)
    if(x>dist)
      @inbounds d_a[x]+=d_a[x-dist]
    end      
    return nothing
end

adder! (generic function with 2 methods)

In [0]:
range=Int(temp)
for i in 0:(range-1)
    dist=2^i
    @cuda blocks=ceil(N/1024) threads=1024 adder!(d_a, dist)
    synchronize()
end
println(d_a)

TypeError: ignored

In [0]:
function sum(a, N)
  for i in 2:N
    a[i]+=a[i-1]
  end
end

sum (generic function with 1 method)

In [0]:
b=Array(d_a)
@test isapprox(b[N], a[N])